## 请设计一个基于DNN模型的推荐系统
阐述两阶段的架构（召回、排序）
以及每个阶段的DNN模型设计：
1. DNN输入层（如何进行特征选择）
2. DNN隐藏层结构
3. DNN输出层

召回阶段 => deep candidate generation model 
排序阶段 => deep ranking model

### 召回阶段：
1. 模型的输入特征有 用户浏览历史、搜索历史、人口统计学信息和其余上下文信息concat生成的输入向量，主要特征的处理：
    + embedded video watches => watch vector，用户的历史观看是一个稀疏的，变长的视频id序列，采用类似于word2vec的做法，每个视频都会被embedding到固定维度的向量中。最终通过加权平均（可根据重要性和时间进行加权）得到固定维度的watch vector
    + embedded search tokens => Search vector，和watch vector生成方式类似
    + 用户画像特征：如地理位置，设备，性别，年龄，登录状态等连续或离散特征都被归一化为[0,1]， 和watch vector以及search vector做拼接（concatenate）
    + 将example age（视频被上传后的时间）作为一个特征拼接到DNN的输入向量。训练时，时间窗口越靠后，该值越接近于0或者为一个小负数。加入了example age特征后，模型效果和观测到的实际数据更加逼近
    + 每个用户的权重需要相等，数据源用网站上所有的视频数据（防止过拟合）
    + 随机抽取用户没有点过的视频作为负样本
    + 上下文的信息是从用户的历史浏览视频来建造，确定某个看过的视频为正样本，它之前连续时间内浏览过的视频作为上文信息特征

2. 模型架构中间是三个隐层的DNN结构。 隐藏层的输入应该是concat好的固定向量，激活函数是Relu系列。

3. 模型的输出分为 训练（离线）和服务（在线），离线训练阶段输出层为softmax层，服务阶段直接用user Embedding和video Embedding计算dot-product表示分数，取topk作为候选结果。最重要问题是在性能。因此使用类似局部敏感哈希LSH（近似最近邻方法）


### 排序阶段：
1. 特征要比召回阶段的更精细，依然需要花费精力将用户及视频数据转化为有效的特征（参考Facebook提出的GBDT+LR模型）
    * 采用embedding的方式映射稀疏离散特征为密集向量，YouTube为每一个类别特征维度生成一个独立的embedding空间（视频id、用户id这种）
    * 连续特征进行归一化、再增加一些特征的非线性表达，例如 对特征进行平方、开方等
    
2. 模型架构中间是三个隐层的DNN结构。 隐藏层的输入应该是concat好的固定向量，激活函数是Relu系列。隐藏层的排列是Tower的方式
3. 模型的输出分为 训练（离线）和服务（在线），离线训练阶段输出层为Weighted LR，得到的是用户对视频的观看期望，即观看的概率乘以视频的时长；服务阶段直接输出softmax结果，选取top N的进行推荐



### summary
1. 召回阶段完成快速筛选（几百万=>几百个），排序阶段完成精排（几百个=>十几个）基于DNN模型完成召回，排序阶段，自动学习item的embedding特征DNN的任务是基于用户信息和上下文环境，来学习用户的embedding向量，模拟矩阵分解的过程，DNN最后一层的输出近似作为用户的特征
2. 特征embedding：
    * 将用户观看过的视频id列表做embedding，取embedding向量的平均值，作为watch vector
    * 把用户搜索过的视频id列表做embedding，取embedding向量的平均值，作为search vector
    * 用户的人口统计学属性做embedding，作为geographic embedding
    * 一些非多值类的特征如性别，还有数值类特征直接做DNN的输入
    * 一些数值类特征，对其进行变换。如对example age进行平方，平方根操作，作为新的特征。

1. 把推荐问题转换成多分类问题，采用Negative Sampling提升模型效果（随机从全量样本中抽取用户没有点击过的item作为label=0，因为推荐列表页中展示的item是算法模型计算出来的 => 用户最有可能会点击的item）
2. 在召回阶段，采用的近似最近邻查找 => 提升效率
3. Youtube的用户对新视频有偏好，引入Example Age（视频上传时间特征） => 与经验分布更Match
4. 不对称的共同浏览问题，采用predicting next watch的方式，利用上文信息，预估下一次浏览的视频 => 从用户的历史视频观看记录中随机拿出来一个作为正样本，然后只用这个视频之前的历史观看记录作为输入
5. 对每个用户提取等数量的训练样本 => 防止一部分非常活跃的用户主导损失函数值
6. 针对某些特征，比如#previous impressions，进行平方和平方根处理，引入3个特征对DNN进行输入 => 简单有效的特征工程，引入了特征的非线性
7. 在优化目标上，没有采用经典的CTR，或者Play Rate，而是采用了每次曝光预期播放时间作为优化目标

## 问题：
1. A/B test 的意思
2. 